In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    print(dirname)
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

---
## I’m Something of a Painter Myself
---
\* All English explanations have been translated

## advance preparation | 사전 준비
---
\- set seed : Increase reproducibility   
\- Create a folder in which GAN-generated images will be stored   
\- I copied the folder and created it separately to use the image separately   
   
\- 시드 설정 : 재현성을 높임   
\- GAN으로 생성된 이미지가 저장될 폴더 만들기   
\- 이미지를 따로 사용하기 위해 폴더를 복사해서 따로 만들었음   

In [ ]:
import numpy as np  # 시드 설정
import tensorflow as tf  # 시드 설정

# 이미지가 저장될 폴더가 없다면 만듭니다.
import os
import shutil

if not os.path.exists("/kaggle/working/tmp"):
    os.makedirs("/kaggle/working/tmp")

if not os.path.exists("/kaggle/working/monet"):
    os.makedirs("/kaggle/working/monet")

if not os.path.exists("/kaggle/working/photo"):
    os.makedirs("/kaggle/working/photo")

if not os.path.exists("/kaggle/working/monet/monet_jpg"):
    shutil.copytree("/kaggle/input/gan-getting-started/monet_jpg", "/kaggle/working/monet/monet_jpg")

if not os.path.exists("/kaggle/working/photo/photo_jpg"):
    shutil.copytree("/kaggle/input/gan-getting-started/photo_jpg", "/kaggle/working/photo/photo_jpg")

# 시드 설정
seed = 3
np.random.seed(seed)
tf.random.set_seed(seed)

print("seed =", seed)

## make generator | 생성자 만들기
---
\- input : Use large enough random vectors as inputs.   
\- Start with a small image and grow in size.   
\- Batch normalization : Normalizes input data. Helps with stable learning.   
\- Passes the generated value to the discriminator.   
\- no compile   
   
\- 입력 : 충분히 큰 랜덤 벡터 사용   
\- 작은 크기로 시작해 크기를 키운다.   
\- 배치 정규화 : 입력 데이터를 정규화한다. 안정적인 학습을 도움.   
\- 판별자에게 생성된 값을 넘긴다.   
\- 생성자는 컴파일 없음.   

In [ ]:
from tensorflow.keras.layers import Dense, Reshape
from tensorflow.keras.layers import BatchNormalization, Activation, LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model

# 생성자 모델을 만듭니다.
generator = Sequential()  # 모델 선언
generator.add(Dense(128 * 32 * 32, input_dim=300, activation=LeakyReLU(0.2)))
'''
512 : 임의로 정한 노드의 수. 충분히 있기만 하면 됨
8 * 8 : 이미지의 최초 크기. 점점 크기를 키울 것이기 때문에 작게 넣음
input_dim=300 : 300차원 크기의 랜덤 벡터를 준비해 넣어라
'''
generator.add(BatchNormalization())  # 배치 정규화 : 입력 데이터를 정규화함. 안정적인 학습을 도움.
generator.add(Reshape((32, 32, 128)))  # 컨볼루션 레이어가 받아들일 수 있는 형태로 바꿈. 인자는 input_shape 형태로 전달
generator.add(UpSampling2D())  # 이미지의 가로 세로 크기를 2배씩 늘림
generator.add(Conv2D(256, kernel_size=5, padding='same'))  # 커널 크기 5 * 5, 패딩으로 크기 유지

generator.add(BatchNormalization())
generator.add(Activation(LeakyReLU(0.2)))
generator.add(UpSampling2D())
generator.add(Conv2D(32, kernel_size=5, padding='same'))

generator.add(BatchNormalization())  # 배치 정규화
generator.add(Activation(LeakyReLU(0.2)))  # 릭키렐루 : 값이 음수이면 0으로 없애지 말고 전달한 값(0.2)을 곱해라.
generator.add(UpSampling2D())  # 크기 2배
generator.add(Conv2D(3, kernel_size=5, padding='same', activation='tanh'))
# 컨볼루션. 이후 판별자에게 값을 넘김, 하이퍼볼릭 탄젠트
# 생성자는 컴파일이 없음
generator.summary()

## make discriminator | 판별자 만들기
---
\- The discriminator should only determine the authenticity and not learn.   
\- It actively reduces the size of the image so that the discriminator can grasp only the characteristics.   
   
\- 판별자는 진위판별만 하고 학습은 하면 안 된다.   
\- 이미지의 크기를 적극적으로 줄여 판별자가 특징만을 파악하게 한다.   

In [ ]:
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model

# 판별자 모델을 만듭니다.
# 진위판별만 하고 학습은 하면 안됨
discriminator = Sequential()  # 모델 선언
discriminator.add(Conv2D(256, kernel_size=5, strides=2, input_shape=(256, 256, 3), padding="same"))
# 노드 128개, 커널 크기 5 * 5, 커널이 2칸씩 이동, 입력 크기 256 * 256, 흑백, 크기 유지 패딩

discriminator.add(Activation(LeakyReLU(0.2)))  # 릭키렐루
discriminator.add(Dropout(0.3))  # 드롭아웃
discriminator.add(Conv2D(128, kernel_size=5, strides=2, padding="same"))
# 노드 128개, 커널 크기 5 * 5, 커널이 2칸씩 이동, 크기 유지 패딩

discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Conv2D(128, kernel_size=5, strides=2, padding="same"))

discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Flatten())  # 플래튼
discriminator.add(Dense(1, activation='sigmoid'))  # 판별 결과 참/거짓

# 판별자 컴파일
discriminator.compile(loss='binary_crossentropy', optimizer='adam')
discriminator.trainable = False  # 학습 금지
discriminator.summary()

## combine generator and discriminator | 생성자와 판별자 합치기
---
\- this is DCGAN.   
   
\- 이것은 DCGAN이다.   

In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

# 생성자와 판별자 모델을 연결시키는 gan 모델을 만듭니다.
ginput = Input(shape=(300,))
# 랜덤한 100개의 벡터를 집어넣어 생성자의 입력 데이터를 만듦
dis_output = discriminator(generator(ginput))
# 생성자 모델에 방금 만든 랜덤 입력 데이터를 넣음, 생성자가 가짜 이미지를 생성하고, 판별자가 진위판별. 그 결과가 변수에 저장됨
gan = Model(ginput, dis_output)
# 생성자와 판별자가 합쳐진 새 모델 선언. 랜덤 입력 데이터(x)와 그것으로 만든 가짜 데이터의 진위여부(y)를 넣어 만듦.
# 생성+판별자 모델 컴파일. 출력은 참 거짓이니 바이너리 사용.
gan.compile(loss='binary_crossentropy', optimizer='adam')
gan.summary()  # 이 모델의 요약정보를 보여주세요

## execute GAN | GAN 실행하기
---
reference : [Load and preprocess images](https://www.tensorflow.org/tutorials/load_data/images)   
   
텐서플로우 공식 문서 참고 : [이미지 로드 및 전처리하기](https://www.tensorflow.org/tutorials/load_data/images?hl=ko)   

In [ ]:
# 신경망을 실행
# 4000번 반복되고(+1을 해 주는 것에 주의), 배치 사이즈는 32, 200번 마다 결과가 저장되게 하였습니다.
epoch = 10000 + 1
batch = 30
saving_interval = 100  # saving_interval : 몇 번에 한 번 중간과정을 저장할까
progress_print_interval = 50  # 학습 과정에서 생성자와 판별자의 오차 출력 간격
img_height = 256
img_width = 256

train_ds = tf.keras.preprocessing.image_dataset_from_directory("/kaggle/working/monet/", seed=seed, image_size=(img_height, img_width), batch_size=batch)
# test_ds = tf.keras.preprocessing.image_dataset_from_directory("/kaggle/working/monet/", validation_split=0.2, subset="validation", seed=seed, image_size=(img_height, img_width), batch_size=batch)
photo_ds = tf.keras.preprocessing.image_dataset_from_directory("/kaggle/working/photo/", seed=seed, image_size=(img_height, img_width), batch_size=batch)

print(type(train_ds))

normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
# normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset=-1)

normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
normal_photo = photo_ds.map(lambda x, y: (normalization_layer(x), y))
# image_batch, labels_batch = next(iter(normalized_ds))
# first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
# print(first_image)

In [ ]:
import matplotlib.pyplot as plt  # 그래프
import seaborn as sns
import PIL
import PIL.Image

ep = []  # 에포크
dl = []  # 판별자 오차
gl = []  # 생성자 오차

true = np.ones((batch, 1))  # batch_size개의 열과 1개의 행을 전부 1로 채운 배열
fake = np.zeros((batch, 1))  # batch_size개의 열과 1개의 행을 전부 0으로 채운 배열

for i in range(3):
    # 실제 데이터를 판별자에 입력하는 부분입니다.
    image_batch, labels_batch = next(iter(normal_photo))
    d_loss_real = discriminator.train_on_batch(image_batch, true)
    
    plt.imshow(image_batch[0].numpy().astype("uint8"), cmap='hsv')  # 이미지가 새까맣게 나오는데 왜 그렇게 되는지는 모르겠다. 이게 GAN이 잘 안되는 이유인 것 같다.
    plt.axis("off")
    plt.show()
    
    # img, label = train_generator.next()
    # d_loss_real = discriminator.train_on_batch(img, true)
    # 판별자에 넣고 학습한다. 입력은 아까 뽑은 이미지, 클래스는 아까 만든 전부 참 배열.
    # train_on_batch : 딱 한 번만 학습시키는 함수

    # 가상 이미지를 판별자에 입력하는 부분입니다.
    noise = np.random.normal(0, 1, (batch, 300))  # 생성자에 집어넣을 랜덤 이미지 만들기
    # 정수가 아니라서 np.random.normal() 함수 사용. 인자의 의미는 아까와 같음.
    # (batch_size, 100) : batch_size만큼 100개의 열을 뽑아달라
    gen_imgs = generator.predict(noise)  # 방금 뽑은 값이 생성자에 들어가 가짜 이미지가 만들어짐
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
    # 판별자에 넣고 학습한다. 입력은 아까 만든 가짜 이미지, 클래스는 아까 만든 전부 거짓 배열.

    # 판별자와 생성자의 오차를 계산합니다.
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    # 진짜와 가짜의 오차 평균이 판별자의 오차
    g_loss = gan.train_on_batch(noise, true)
    # 이건 생성자의 오차.

    # 학습 진행하는 동안 생성자와 판별자의 오차 출력
    if i % progress_print_interval == 0:
        print('epoch:%d' % i, ' d_loss:%.4f' % d_loss, ' g_loss:%.4f' % g_loss)

    # 생성자와 판별자의 오차 값을 저장해 데이터 프레임으로 만들고 그래프 그리기
    ep.append(i)
    dl.append(d_loss)
    gl.append(g_loss)

    # 이부분은 중간 과정을 이미지로 저장해 주는 부분입니다.
    if i % saving_interval == 0:
        # r, c = 5, 5
        noise = np.random.normal(0, 1, (25, 300))
        gan_imgs = generator.predict(noise)

        # Rescale images 0 - 1
        gan_imgs = 0.5 * gan_imgs + 0.5

        fig, axs = plt.subplots(5, 5)
        count = 0
        for j in range(5):
            for k in range(5):
                axs[j, k].imshow(gan_imgs[count, :, :, 0], cmap='hsv')
                axs[j, k].axis('off')
                count += 1
        plt.show()
        fig.savefig("tmp/photomonet_%d.png" % i)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

result_data = {
    'epoch': ep, 
    'd_loss': dl, 
    'g_loss': gl
}

history = pd.DataFrame(result_data)

sns.set(rc={'figure.figsize':(16, 9)})

sns.set_style("white")
x = history['epoch']
y_1 = history['d_loss']

sns.lineplot(x=x, y=y_1, label='d_loss', color='blue', alpha=0.7)

plt.xlabel('epoch', fontsize=20)
plt.ylabel('loss', fontsize=20)
plt.title('discriminator loss per epoch', fontsize=25)

plt.grid()
plt.legend(loc='upper left')
plt.show()


sns.set_style("white")
x = history['epoch']
y_2 = history['g_loss']

sns.lineplot(x=x, y=y_2, label='g_loss', color='green', alpha=0.7)

plt.xlabel('epoch', fontsize=20)
plt.ylabel('loss', fontsize=20)
plt.title('generator loss per epoch', fontsize=25)

plt.grid()
plt.legend(loc='upper left')
plt.show()